In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandarallel import pandarallel
pandarallel.initialize(shm_size_mb=4000, progress_bar=True)

In [ ]:
df = pd.read_csv("/home/fvalle/phd/datasets/merged/mainTable.csv", index_col=0)
df_files = pd.read_csv("/home/fvalle/phd/datasets/merged/files.dat", index_col=0)

In [ ]:
#fpkm threshholds
df = df.where(df<1e5,1e5)
df = df.where(df>1e-1,0)

In [ ]:
def get_site_columns(site='Breast'):
    return df_files[df_files['site']==site].index.values

def get_not_site_columns(site='Breast'):
    return df_files[df_files['site']!=site].index.values

In [ ]:
gene = df.index[130]

In [ ]:
HDE_genes = []
sigmadiff = []

In [ ]:
for gene in df.index:
    print(gene)
    bins=np.linspace(1e-1,df.loc[gene,:].max(),100)
    T_data = df[get_site_columns()].loc[gene,:].astype(float).values
    NT_data = df[get_not_site_columns()].loc[gene,:].astype(float).values
    T_mean = T_data.mean()
    NT_mean = NT_data.mean()
    NT_sigma = NT_data.std()
    sigmadiff.append([gene, np.abs(NT_mean-T_mean)/NT_sigma])
    if np.abs(NT_mean-T_mean) > 0.5*NT_sigma:
        fig=plt.figure()
        plt.title(gene)
        nT,_,_ = plt.hist(T_data, density=True, histtype='step', bins=bins)
        nNT,_,_ = plt.hist(NT_data, density=True, histtype='step', bins=bins)
        plot_max = np.where(nT>nNT,nT,nNT).max()
        plt.vlines([T_mean, NT_mean],0,plot_max, linestyles='dashed', colors=['b','orange'])
        plt.vlines([NT_mean-NT_sigma, NT_mean+NT_sigma],0, plot_max, colors='red')
        plt.tick_params(labelsize=18)
        plt.xticks(rotation=90)
        plt.show()
        HDE_genes.append(gene)

In [ ]:
means = df.parallel_apply(lambda x: x[get_not_site_columns()].mean(), axis=1)
variances = df.parallel_apply(lambda x: x[get_not_site_columns()].var(), axis=1)
means_T = df.parallel_apply(lambda x: x[get_site_columns()].mean(), axis=1)
variances_T = df.parallel_apply(lambda x: x[get_site_columns()].var(), axis=1)

In [ ]:
df_info = pd.DataFrame(index=df.index)
df_info.insert(0,'mean', means)
df_info.insert(1,'std', variances.apply(np.sqrt))
df_info.insert(2,'mean_T', means_T)
df_info.insert(3,'std_T', variances_T.apply(np.sqrt))
df_info.insert(4,'(mean-mean_T)/std', (means-means_T).apply(np.abs)/variances.apply(np.sqrt))
df_info.insert(5,'cv2_T', variances_T/means_T/means_T)
df_info.insert(6,'cv2', variances/means/means)
df_info = df_info.sort_values(by='(mean-mean_T)/std', ascending=False)

In [ ]:
#df_info.to_csv("df_info.csv", index=True, header=True)
df_info = pd.read_csv("df_info.csv", index_col=0)
df_info

In [ ]:
HDE_genes = df_info.index.values

In [ ]:
fig, ax=plt.subplots()
df.loc['CELA3A',get_site_columns()].hist(density=True, ax=ax, bins=np.linspace(0,10000,10), histtype='step')
df.loc['CELA3A',get_not_site_columns()].hist(density=True, ax=ax, bins=np.linspace(0,10000,10), histtype='step')

In [ ]:
plt.scatter(df_info['mean'], df_info['mean_T'])
plt.xscale('log')
plt.yscale('log')

In [ ]:
plt.scatter(df_info['mean_T'],df_info['cv2_T'])
plt.plot(df_info['mean_T'],1./df_info['mean_T'])
plt.xscale('log')
plt.yscale('log')
plt.ylim(1e-3,1e6)
plt.xlim(1e-2,1e5)

In [ ]:
with open("HDE_Breast.txt", 'w') as f:
    for g in HDE_genes:
        f.write("%s\n"%g)

In [ ]:
HDE_genes